# Singularly Perturbed Equations

## Boundary layers
Consider the linear 2-point boundary value problem
$$ \begin{array}{c}a u'(x) = \kappa u''(x) + \psi(x), \\
 u(0) = \alpha, \quad u(1) = \beta. \end{array}$$
    If $a$ is small relative to $\kappa$􏰭, then this problem is easy to solve.  Now, suppose $a$ is large relative to $\kappa$.  Here, we introduce a parameter $\epsilon = \frac{\kappa}{a}$, and rewrite the above equation in the form
$$ \epsilon u''(x) - u'(x) = f(x).$$
Then taking $a$ large relative to $\kappa$ corresponds to the case $\epsilon \ll 􏰱1$.􏰭 

__Example__:
Considering
$$ \begin{array}{c}\epsilon u''(x) - u'(x) = f(x), \\
 u(0) = \alpha, \quad u(1) = \beta. \end{array}$$
Here, $\alpha = 1$, $\beta=3$, and $f(x) = -1$.  In this case the exact solution (why?) is
$$ u(x) = \alpha + x + \beta + (\beta - \alpha -1)\left(\frac{e^{x/\epsilon} -1}{e^{1/\epsilon}-1}\right) $$


In [4]:
#%matplotlib notebook
%matplotlib inline

# environment setting, before any codes
import numpy as np
import scipy.linalg as slinalg

import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output, display

In [ ]:
def exact_fun(x, alpha, beta, epsilon):
    return alpha + x + (beta - alpha - 1)*((np.exp(x/epsilon)-1)/(np.exp(1/epsilon)-1))

alpha = 1
beta = 3
epsilon = 0.002

number = 101
def draw_fun(number, alpha, beta, epsilon):
    x = np.linspace(0, 1, number)
    y = exact_fun(x, alpha, beta, epsilon)
    plt.plot(x,y,'r',linewidth=2)
    return epsilon
#draw_fun(number, alpha, beta, epsilon)
w = interactive(draw_fun, number=fixed(number), alpha=fixed(alpha), beta=fixed(beta),\
                epsilon=widgets.FloatSlider(min=0.01,max=1.00,step=0.01, value=1.00, description='$\epsilon:$',))
display(w)

Note that as $\epsilon \rightarrow 0$􏰫 the solution tends toward a discontinuous function that jumps to the value $\beta$ at the last possible moment.  This region of rapid transition is called the _boundary layer_ and it can be shown that for this problem the width of this layer is $\mathcal{O}(\epsilon)$ as $\epsilon \rightarrow 0$.

The above equation with $0 < \epsilon \ll 1$ is called a _singularly perturbed equation_. It is a small perturbation of equation $-u'(x) = f(x)$, but this small perturbation completely changes the character of the equation (from a first order to a second order equation). Typically any differential equation having a small parameter multiplying the highest order derivative will give a singular perturbation problem.

### Finite Difference Methods
This _singularly perturbed equation_ can be discretized to second order by
$$ \epsilon\left(\frac{U_{i+1} - 2U_i + U_{i-1}}{h^2}\right) - \left(\frac{U_{i+1} - U_{i-1}}{2h}\right) = f_i, \qquad i = 1,\ldots, m,$$
with $U_0 = \alpha$ and $U_{m+1} = \beta$.  This gives the linear system $AU = F$, where $U$ is the vector of unknowns $U = [U_0, U_1, \ldots, U_m, U_{m+1}]^T$, $A$ is the tridiagonal matrix
$$
    A = \frac{1}{h^2}\left[\begin{array}{cccccccc}
    h^2  &  0 &   &    &   &    & & \\
    \epsilon + h/2  & -2\epsilon &  \epsilon - h/2 &    &   &    & &\\
       &  \epsilon + h/2  & -2\epsilon &  \epsilon - h/2 &   &    & &\\ 
       &    &  \epsilon + h/2  & -2\epsilon &  \epsilon - h/2 &    & &\\
       &    & & \ddots & \ddots &\ddots & &\\
       &    &    &    & \epsilon + h/2  & -2\epsilon &  \epsilon - h/2  &\\
       &    &    &    &   &  \epsilon + h/2  & -2\epsilon &  \epsilon - h/2 \\
       &    &    &    &   &   &  0 & h^2\end{array}\right],
           \quad
   \text{and}\quad F = \left[\begin{array}{c} \alpha \\ f_1\\ f_2 \\ f_3 \\ \vdots \\ f_{m-1} \\ f_m \\ \beta\end{array}\right]. 
$$

In [ ]:
def generate_grid(left, right, m):
    h = (right - left)/(m+1)
    x = np.zeros(m+2)
    for j in range(m+2):
        x[j] = j*h
    return  x, h

def generate_A_interior(epsilon, h, m):
    A = np.zeros([m+2, m+2])
    for i in range(1,m+1):
        A[i, i-1] = epsilon + h/2.
        A[i, i] = -2*epsilon
        A[i, i+1] = epsilon - h/2
    #A[1:m+1,1:m+1] = - 2*np.eye(m, m) 
    #A[ += np.eye(m+2, m+2, -1) + np.eye(m+2, m+2, 1)
    return A/h**2

def generate_F_interior(h, m, x, fun):
    F = np.zeros(m+2)
    for i in range(1,m+1):
        F[i] = fun(x[i])
    return F

def Dirichlet_bc(A, F, alpha, beta, h, fun):
    A[0,0] = 1
    A[-1,-1] = 1
    F[0] = alpha
    F[-1] = beta
    return A, F

def fun(x):
    return -1

def FDM_BVP(m, left, right, alpha, beta, epsilon, bc, fun, exact_fun):
    # generate the grid
    x, h = generate_grid(left, right, m)
    #
    U = np.zeros(m+2)
    # 
    A = generate_A_interior(epsilon, h, m)
    #print (A)
    F = generate_F_interior(h, m, x, fun)
    #print (F)
    # bc
    A, F = bc(A, F, alpha, beta, h, fun)
    #print (A)
    #print (F)
    U = np.linalg.solve(A, F)
    
    u = exact_fun(x, alpha, beta, epsilon)
    return np.max(np.abs(U-u)), x, U, u

In [ ]:
alpha = 1
beta = 3
epsilon = 0.01
error, x, U, u = FDM_BVP(9, 0, 1, alpha, beta, epsilon, Dirichlet_bc, fun, exact_fun)

print ("max norm error: %7.2e" % error )
# let us plot the approximation solution in blue, and the exact solution in red
#plt.plot(x, u, 'ro-')
plt.plot(x, U, 'bo-')
number = 101
draw_fun(number, alpha, beta, epsilon)

In [ ]:
def draw_appximation(m, epsilon):
    error, x, U, u = FDM_BVP(m, 0, 1, alpha, beta, epsilon, Dirichlet_bc, fun, exact_fun)
    print ("max norm error: %7.2e" % error )
    # let us plot the approximation solution in blue, and the exact solution in red
    #plt.plot(x, u, 'ro-')
    plt.plot(x, U, 'bo-')
    number = 101
    draw_fun(number, alpha, beta, epsilon)
    
#draw_appximation(9, 0.01)
w = interactive(draw_appximation, m=widgets.IntSlider(min=9,max=2000,step=10, value=9, description='$m:$',),\
                epsilon=widgets.FloatSlider(min=0.01,max=1.00,step=0.01, value=1.00, description='$\epsilon:$',))
display(w)

In [ ]:
levels = 8
m = np.array([9, 19, 39, 79, 159, 319, 639, 1279], int)
err_max = np.zeros(levels)
print ('   h      max error   order  ')
for i in range (levels):
    err_max[i],_,_,_ = FDM_BVP(m[i], 0, 1, alpha, beta, epsilon, Dirichlet_bc, fun, exact_fun)
    if (i == 0):
        print ('%7.2e  %7.2e' % (1/(m[i]+1), err_max[i]))
    else:
        print ('%7.2e  %7.2e    %4.2f' % (1/(m[i]+1), err_max[i], \
                                        np.log(err_max[i-1]/err_max[i])/np.log((m[i]+1)/(m[i-1]+1))))

## Nonuniform Mesh 

### A general approach to deriving the coefficients
For computing the finite difference coefficients for computing an approximation to $u^{(k)}(\bar{x})$, the $k$th derivative of $u(x)$ evaluated at $\bar{x}$, based on an arbitrary stencil of $n \geq 􏰇k+1$ points $x_1, \ldots, x_n$. We assume $u(x)$ is sufficiently smooth, so that the Taylor series expansions of $u$ at each point $x_i$ in the stencil about $u(\bar{x})$ yield
$$u\left(x_{i}\right)=u(\bar{x})+\left(x_{i}-\bar{x}\right) u^{\prime}(\overline{x})+\cdots+\frac{1}{k !}\left(x_{i}-\bar{x}\right)^{k} u^{(k)}(\bar{x})+\cdots$$
for $i = 1,\ldots, n$.  We want to find a linear combination of these values that agrees with $u^{(k)}(\bar{x})$ as well as possible. So we want
$$ c_{1} u\left(x_{1}\right)+c_{2} u\left(x_{2}\right)+\cdots+c_{n} u\left(x_{n}\right)=u^{(k)}(\bar{x})+O\left(h^{p}\right) $$
where $p$ is as large as possible. (Here $h$ is some measure of the width of the stencil)

In [ ]:
#import math
def fdcoeff(k, xbar, x):
    n = np.size(x)
    if (n < k+1): 
        print ("Try again!")
        exit
    A = np.ones([n,n])
    xrow = x - xbar
    for i in range(1,n):
        A[i,:] = xrow**(i-1)/math.factorial(i-1)       

In [ ]:
fdcoeff(1, 0, [-1,0,1])